# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [94]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [95]:
# your code here
orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [96]:
orders.shape

(397924, 14)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [97]:
# your code here

In [98]:
orders.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent'],
      dtype='object')

In [99]:
customer_spending = orders.groupby('CustomerID').agg({'amount_spent': 'sum'})


#display the results
customer_spending.head(10)

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
12352,2506.04
12353,89.00
12354,1079.40
12355,459.40


In [100]:


amount_spent_by_customer = orders.groupby('CustomerID')['amount_spent'].sum()


quantiles = amount_spent_by_customer.quantile([0.1, 0.3, 0.5, 0.7, 0.75, 0.95, 1.0])

# Select customers within the desired quantile ranges
customers_in_10_percentile = amount_spent_by_customer[amount_spent_by_customer <= quantiles[0.1]]
customers_in_30_percentile = amount_spent_by_customer[(amount_spent_by_customer > quantiles[0.1]) & (amount_spent_by_customer <= quantiles[0.3])]
customers_in_50_percentile = amount_spent_by_customer[(amount_spent_by_customer > quantiles[0.3]) & (amount_spent_by_customer <= quantiles[0.5])]
customers_in_70_percentile = amount_spent_by_customer[(amount_spent_by_customer > quantiles[0.5]) & (amount_spent_by_customer <= quantiles[0.7])]
customers_above_75_percentile = amount_spent_by_customer[amount_spent_by_customer > quantiles[0.75]]
customers_above_95_percentile = amount_spent_by_customer[amount_spent_by_customer > quantiles[0.95]]



In [101]:
customers_in_10_percentile

CustomerID
12353     89.00
12401     84.30
12445    133.40
12506     73.50
12512    138.03
          ...  
18255    103.30
18262    149.48
18268     25.50
18277    110.38
18281     80.82
Name: amount_spent, Length: 434, dtype: float64

In [102]:
customers_above_70_percentile

CustomerID
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12352     2506.04
           ...   
18259     2338.60
18260     2643.20
18272     3078.58
18283     2094.88
18287     1837.28
Name: amount_spent, Length: 1301, dtype: float64

In [105]:


def assign_ranking(amount_spent):
    if amount_spent >= quantiles[0.75]:
        return 'VIP'
    elif amount_spent >= quantiles[0.75] and amount_spent <= quantiles[0.95] :
        return 'Preferred'
    else:
        return 'Normal'

orders['Ranking'] = orders['CustomerID'].map(amount_spent_by_customer).apply(assign_ranking)  # add closing parenthesis


In [104]:

def assign_ranking(amount_spent):
    if amount_spent >= quantiles[0.75]:
        return 'VIP'
    elif amount_spent >= quantiles[0.75] & amount_spent <= quantiles[0.95] :
        return  'Preferred'
    else:
        return 'Normal'


orders['Ranking'] = orders['CustomerID'].map(amount_spent_by_customer).apply(assign_ranking)



TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [106]:
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Ranking
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,VIP
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,VIP
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20,Normal
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60,Normal
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60,Normal
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60,Normal


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [107]:
# your code here

In [108]:
vip_customers = orders.loc[orders['Ranking'] == 'VIP'].groupby('Country')['CustomerID'].nunique()

In [109]:
vip_customers

Country
Australia            4
Austria              3
Belgium             12
Canada               1
Channel Islands      4
Cyprus               4
Denmark              3
EIRE                 3
Finland              5
France              29
Germany             39
Greece               1
Iceland              1
Israel               2
Italy                5
Japan                4
Lebanon              1
Malta                1
Netherlands          1
Norway               7
Poland               1
Portugal             7
Singapore            1
Spain                9
Sweden               2
Switzerland          9
United Kingdom     932
Name: CustomerID, dtype: int64

In [110]:
Country_Rank= vip_customers.sort_values(ascending=False)

In [111]:
Country_Rank.head()

Country
United Kingdom    932
Germany            39
France             29
Belgium            12
Switzerland         9
Name: CustomerID, dtype: int64

# The United Kingdom has the highest numbers of VIP customers

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [112]:
# your code here

In [113]:
top_countries = orders.loc[orders['Ranking'].isin(['VIP', 'Preferred'])].groupby('Country')['CustomerID'].nunique()

top_countries_sorted = top_countries.sort_values(ascending=False)



In [114]:
top_countries_sorted

Country
United Kingdom     932
Germany             39
France              29
Belgium             12
Switzerland          9
Spain                9
Portugal             7
Norway               7
Italy                5
Finland              5
Japan                4
Australia            4
Cyprus               4
Channel Islands      4
Austria              3
EIRE                 3
Denmark              3
Sweden               2
Israel               2
Iceland              1
Greece               1
Lebanon              1
Malta                1
Netherlands          1
Poland               1
Singapore            1
Canada               1
Name: CustomerID, dtype: int64

# The United Kingdom has the highest numbers of VIP customers and Preferred customers